In [249]:
from six.moves import urllib
import numpy as np
urllib.request.urlretrieve('https://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data','datasets/mpg.data')

('datasets/mpg.data', <http.client.HTTPMessage at 0x7f1701b1a940>)

In [250]:
import pandas as pd
df = pd.read_table('datasets/mpg.data',delim_whitespace = True ,header = None)
df.columns = ['mpg','cylinder','displacement','horsepower','weight','acceleration','year','origin','carname']

In [251]:
data = df.loc[~(df['horsepower']== '?')]
data['horsepower'] = data['horsepower'].apply(pd.to_numeric)
data = data.iloc[:,:8]


/home/siva/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [252]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(data.iloc[:,1:],data.iloc[:,1],test_size = 0.2,random_state = 42)


In [253]:
from sklearn.base import BaseEstimator, TransformerMixin

class Attributeselector(BaseEstimator, TransformerMixin):
        def __init__(self,attributes):
            self.attributes = attributes
        def fit(self,X):
            return self
        def transform(self,X):
            return X[self.attributes].values

In [254]:
num_attributes = ['displacement','horsepower','weight','acceleration']
cat_attributes = ['cylinder','year','origin']


In [255]:
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
                            ('select',Attributeselector(num_attributes)),
                            
                            ('scaler',StandardScaler()),
                        ])





In [256]:
cat_pipeline = Pipeline([
        
                            ('select',Attributeselector(cat_attributes)),
                        ])



In [257]:


combined_pipeline = FeatureUnion( transformer_list = [
                                    ('num',num_pipeline),
        
                                    ('cat',cat_pipeline),
                                ])

In [258]:
processed_train = combined_pipeline.fit_transform(x_train)

processed_test = combined_pipeline.fit_transform(x_test)


In [259]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(processed_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [260]:
from sklearn.metrics import mean_squared_error

predictions = lr.predict(processed_test)
error = mean_squared_error(predictions,y_test)
RMSE = np.sqrt(error)
print("predictions:\t",predictions[10:30])
print("labels:\t",list(y_test[10:30]))

predictions:	 [ 8.  6.  8.  4.  6.  4.  6.  4.  4.  4.  6.  4.  4.  8.  4.  4.  4.  6.
  4.  4.]
labels:	 [8, 6, 8, 4, 6, 4, 6, 4, 4, 4, 6, 4, 4, 8, 4, 4, 4, 6, 4, 4]
